In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv('../assets/patient/patient_concat.csv', index_col=0)
df

,message,message_type
0,Hi,patient_greet
1,Hello,patient_greet
2,Hey there,patient_greet
3,Good morning,patient_greet
4,Good afternoon,patient_greet
...,...,...
450,"Unfortunately, I have a conflicting schedule.",patient_rejection
451,"Sorry, that time doesn't suit my availability.",patient_rejection
452,I need to cancel as I have another appointment.,patient_rejection
453,"Unfortunately, I won't be available at that time.",patient_rejection


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import joblib

tfidf_vectorizer = TfidfVectorizer(lowercase=True)
X_full = df['message'].sort_index().values
y_full = df['message_type'].sort_index().values
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['message_type'], test_size=0.2, random_state=42)
message_tfidf = tfidf_vectorizer.fit_transform(X_train)
joblib.dump(tfidf_vectorizer, '../assets/nn_vectorizer.joblib')

['../assets/nn_vectorizer.joblib']

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_train_categorical = to_categorical(y_train_encoded)
y_test_encoded = le.fit_transform(y_test)
y_test_categorical = to_categorical(y_test_encoded)

x_test = tfidf_vectorizer.transform(X_test)

model = Sequential()

model.add(Dense(units=50, activation='relu', input_shape=(message_tfidf.shape[1],)))

model.add(Dense(units=30, activation='relu'))

model.add(Dense(units=20, activation='relu'))

model.add(Dense(len(le.classes_), activation='softmax'))

In [15]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','categorical_crossentropy' ,'mean_squared_error', 'mean_absolute_error'])

with tf.device('/gpu:0'):
    model.fit(message_tfidf.toarray(), y_train_categorical, epochs=30, validation_data=(x_test.toarray(), y_test_categorical)) 

model.save('../assets/neural_network.h5')
joblib.dump(le, '../assets/label_encoder.joblib')

Epoch 1/30
12/12 [==============================] - 1s 20ms/step - loss: 1.3810 - accuracy: 0.3709 - categorical_crossentropy: 1.3810 - mean_squared_error: 0.1868 - mean_absolute_error: 0.3743 - val_loss: 1.3622 - val_accuracy: 0.6154 - val_categorical_crossentropy: 1.3622 - val_mean_squared_error: 0.1845 - val_mean_absolute_error: 0.3719
Epoch 2/30
12/12 [==============================] - 0s 7ms/step - loss: 1.3453 - accuracy: 0.7335 - categorical_crossentropy: 1.3453 - mean_squared_error: 0.1823 - mean_absolute_error: 0.3697 - val_loss: 1.3182 - val_accuracy: 0.8352 - val_categorical_crossentropy: 1.3182 - val_mean_squared_error: 0.1788 - val_mean_absolute_error: 0.3661
Epoch 3/30
12/12 [==============================] - 0s 8ms/step - loss: 1.2846 - accuracy: 0.8956 - categorical_crossentropy: 1.2846 - mean_squared_error: 0.1745 - mean_absolute_error: 0.3615 - val_loss: 1.2362 - val_accuracy: 0.8571 - val_categorical_crossentropy: 1.2362 - val_mean_squared_error: 0.1680 - val_mean_ab

['../assets/label_encoder.joblib']